In [10]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load earthquake dataset (update file path)
file_path = r"C:\Users\harsh\Downloads\Indian_earthquake_data.csv"
df = pd.read_csv(file_path)

# Drop unnecessary columns (if any)
df = df.drop(columns=["Origin Time"])  # Removing timestamps as they are not useful directly

# Encode categorical 'Location' feature
le = LabelEncoder()
df["Location"] = le.fit_transform(df["Location"])

# Define features and target
X = df[["Latitude", "Longitude", "Depth", "Location"]]  # Features
y = df["Magnitude"]  # Target variable (Magnitude prediction)

# Split dataset into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest Regressor for earthquake magnitude prediction
model = RandomForestRegressor(n_estimators=500, random_state=42)
model.fit(X_train_scaled, y_train)

# Save the trained model
joblib.dump(model, "earthquake_model.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(le, "location_encoder.pkl")

print("Model training complete! Model saved as 'earthquake_model.pkl'")


Model training complete! Model saved as 'earthquake_model.pkl'


In [ ]:
import requests
import pandas as pd
import numpy as np
import joblib

# Load the trained earthquake prediction model
model = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_prediction/earthquake_model.pkl")

# Load the pre-trained scaler
scaler = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_prediction/scaler.pkl")

# Load the location encoder (since "Location" is missing)
location_encoder = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_prediction/location_encoder.pkl")

def get_earthquake_data():
    """Fetch the latest earthquake data from the USGS Earthquake API."""
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&limit=1&orderby=time"
    response = requests.get(url).json()

    if "features" in response and len(response["features"]) > 0:
        earthquake = response["features"][0]
        properties = earthquake["properties"]
        geometry = earthquake["geometry"]["coordinates"]

        earthquake_data = {
            "Latitude": geometry[1],
            "Longitude": geometry[0],
            "Depth": geometry[2],
            "Magnitude": properties["mag"],
            "Location": properties["place"]  # ✅ Add the missing "Location"
        }
        return earthquake_data

    return None

def predict_earthquake():
    """Predict earthquake risk using real-time earthquake data."""
    earthquake_data = get_earthquake_data()

    if earthquake_data:
        print("\n--- Real-Time Earthquake Data ---")
        for key, value in earthquake_data.items():
            print(f"{key}: {value}")

        # Encode "Location" using the same encoder used during training
        encoded_location = location_encoder.transform([earthquake_data["Location"]])[0]

        # Prepare input data (Including Location now)
        input_data = pd.DataFrame([[earthquake_data["Latitude"], earthquake_data["Longitude"], earthquake_data["Depth"], encoded_location]],
                                  columns=["Latitude", "Longitude", "Depth", "Location"])
        
        # Scale the input data
        input_scaled = scaler.transform(input_data)

        # Predict earthquake magnitude
        predicted_magnitude = model.predict(input_scaled)[0]

        # Interpret earthquake risk level
        if predicted_magnitude >= 6.0:
            risk_level = "High Earthquake Risk"
        elif predicted_magnitude >= 4.0:
            risk_level = "Moderate Earthquake Risk"
        else:
            risk_level = "Low Earthquake Risk"

        print("\n--- Earthquake Prediction ---")
        print(f"Predicted Magnitude: {predicted_magnitude:.2f}")
        print(f"⚠ Earthquake Risk Level: {risk_level}")

    else:
        print("Failed to retrieve earthquake data.")

# Run the prediction function
predict_earthquake()



--- Real-Time Earthquake Data ---
Latitude: 38.7963333129883
Longitude: -122.738334655762
Depth: 2.09999990463257
Magnitude: 0.86
Location: 3 km NE of The Geysers, CA


ValueError: y contains previously unseen labels: '3 km NE of The Geysers, CA'

In [16]:
import requests
import pandas as pd
import numpy as np
import joblib

# Load the trained earthquake prediction model
model = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_prediction/earthquake_model.pkl")

# Load the pre-trained scaler
scaler = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_prediction/scaler.pkl")

# Load the location encoder (since "Location" is missing)
location_encoder = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_prediction/location_encoder.pkl")

def get_earthquake_data():
    """Fetch the latest earthquake data from the USGS Earthquake API."""
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&limit=1&orderby=time"
    response = requests.get(url).json()

    if "features" in response and len(response["features"]) > 0:
        earthquake = response["features"][0]
        properties = earthquake["properties"]
        geometry = earthquake["geometry"]["coordinates"]

        earthquake_data = {
            "Latitude": geometry[1],
            "Longitude": geometry[0],
            "Depth": geometry[2],
            "Magnitude": properties["mag"],
            "Location": properties["place"]  # ✅ Location is included
        }
        return earthquake_data

    return None

def predict_earthquake():
    """Predict earthquake risk using real-time earthquake data."""
    earthquake_data = get_earthquake_data()

    if earthquake_data:
        print("\n--- Real-Time Earthquake Data ---")
        for key, value in earthquake_data.items():
            print(f"{key}: {value}")

        # Try to encode the location, assign 0 if it's unseen
        try:
            encoded_location = location_encoder.transform([earthquake_data["Location"]])[0]
        except ValueError:
            print(f"⚠ Warning: Unseen location '{earthquake_data['Location']}' detected. Assigning default encoding.")
            encoded_location = 0  # Assign a default value for unknown locations

        # Prepare input data (Including Location)
        input_data = pd.DataFrame([[earthquake_data["Latitude"], earthquake_data["Longitude"], earthquake_data["Depth"], encoded_location]],
                                  columns=["Latitude", "Longitude", "Depth", "Location"])
        
        # Scale the input data
        input_scaled = scaler.transform(input_data)

        # Predict earthquake magnitude
        predicted_magnitude = model.predict(input_scaled)[0]

        # Interpret earthquake risk level
        if predicted_magnitude >= 6.0:
            risk_level = "High Earthquake Risk"
        elif predicted_magnitude >= 4.0:
            risk_level = "Moderate Earthquake Risk"
        else:
            risk_level = "Low Earthquake Risk"

        print("\n--- Earthquake Prediction ---")
        print(f"Predicted Magnitude: {predicted_magnitude:.2f}")
        print(f"⚠ Earthquake Risk Level: {risk_level}")

    else:
        print("Failed to retrieve earthquake data.")

# Run the prediction function
predict_earthquake()



--- Real-Time Earthquake Data ---
Latitude: 38.7963333129883
Longitude: -122.738334655762
Depth: 2.09999990463257
Magnitude: 0.86
Location: 3 km NE of The Geysers, CA
⚠ Warning: Unseen location '3 km NE of The Geysers, CA' detected. Assigning default encoding.

--- Earthquake Prediction ---
Predicted Magnitude: 3.70
⚠ Earthquake Risk Level: Low Earthquake Risk


In [18]:
import requests
import pandas as pd
import numpy as np
import joblib

# Load the trained earthquake prediction model
model = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_prediction/earthquake_model.pkl")

# Load the pre-trained scaler
scaler = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_prediction/scaler.pkl")

# Load the location encoder
location_encoder = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_prediction/location_encoder.pkl")

def get_earthquake_data(location):
    """Fetch earthquake data (latitude, longitude, depth, magnitude) from USGS API."""
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&limit=10&orderby=time"
    response = requests.get(url).json()

    if "features" in response and len(response["features"]) > 0:
        for earthquake in response["features"]:
            properties = earthquake["properties"]
            geometry = earthquake["geometry"]["coordinates"]
            place = properties["place"]

            # Check if location matches user input
            if location.lower() in place.lower():
                return {
                    "Latitude": geometry[1],
                    "Longitude": geometry[0],
                    "Depth": geometry[2],
                    "Magnitude": properties["mag"],
                    "Location": place
                }

    return None

def predict_earthquake():
    """Predict earthquake risk using real-time earthquake data."""
    location = input("Enter Location Name: ")

    earthquake_data = get_earthquake_data(location)

    if earthquake_data:
        # Print real-time earthquake data in requested format
        print("\n--- Real-Time Earthquake Data ---")
        print(f"Latitude: {earthquake_data['Latitude']}")
        print(f"Longitude: {earthquake_data['Longitude']}")
        print(f"Depth: {earthquake_data['Depth']}")
        print(f"Magnitude: {earthquake_data['Magnitude']}")
        print(f"Location: {earthquake_data['Location']}")

        # Try to encode the location, assign 0 if it's unseen
        try:
            encoded_location = location_encoder.transform([earthquake_data["Location"]])[0]
        except ValueError:
            print(f"⚠ Warning: Unseen location '{earthquake_data['Location']}' detected. Assigning default encoding.")
            encoded_location = 0  # Assign a default value for unknown locations

        # Prepare input data
        input_data = pd.DataFrame([[earthquake_data["Latitude"], earthquake_data["Longitude"], earthquake_data["Depth"], encoded_location]],
                                  columns=["Latitude", "Longitude", "Depth", "Location"])

        # Scale the input data
        input_scaled = scaler.transform(input_data)

        # Predict earthquake magnitude
        predicted_magnitude = model.predict(input_scaled)[0]

        # Interpret earthquake risk level
        if predicted_magnitude >= 6.0:
            risk_level = "High Earthquake Risk"
        elif predicted_magnitude >= 4.0:
            risk_level = "Moderate Earthquake Risk"
        else:
            risk_level = "Low Earthquake Risk"

        # Print earthquake prediction in requested format
        print("\n--- Earthquake Prediction ---")
        print(f"Predicted Magnitude: {predicted_magnitude:.2f}")
        print(f"⚠ Earthquake Risk Level: {risk_level}")

    else:
        print("❌ No recent earthquakes found for this location.")

# Run the prediction function
predict_earthquake()


❌ No recent earthquakes found for this location.


In [21]:
import requests
import pandas as pd
import numpy as np
import joblib

# Load the trained earthquake prediction model
model = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_prediction/earthquake_model.pkl")

# Load the pre-trained scaler
scaler = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_prediction/scaler.pkl")

# Load the location encoder
location_encoder = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_prediction/location_encoder.pkl")

def get_earthquake_data(location):
    """Fetch earthquake data (latitude, longitude, depth, magnitude) from USGS API."""
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&limit=10&orderby=time"
    response = requests.get(url).json()

    print("🔍 API Response:", response)  # Debugging Print

    if "features" in response and len(response["features"]) > 0:
        for earthquake in response["features"]:
            properties = earthquake["properties"]
            geometry = earthquake["geometry"]["coordinates"]
            place = properties["place"]

            if location.lower() in place.lower():
                return {
                    "Latitude": geometry[1],
                    "Longitude": geometry[0],
                    "Depth": geometry[2],
                    "Magnitude": properties["mag"],
                    "Location": place
                }

    print("❌ No recent earthquakes found for this location.")
    return None

def predict_earthquake():
    """Predict earthquake risk using real-time earthquake data."""
    location = input("Enter Location Name: ")

    earthquake_data = get_earthquake_data(location)

    if earthquake_data:
        print("\n--- Real-Time Earthquake Data ---")
        for key, value in earthquake_data.items():
            print(f"{key}: {value}")  

        try:
            encoded_location = location_encoder.transform([earthquake_data["Location"]])[0]
        except ValueError:
            print(f"⚠ Warning: Unseen location '{earthquake_data['Location']}' detected. Assigning default encoding.")
            encoded_location = 0  

        input_data = pd.DataFrame([[earthquake_data["Latitude"], earthquake_data["Longitude"], earthquake_data["Depth"], encoded_location]],
                                  columns=["Latitude", "Longitude", "Depth", "Location"])

        print("📊 Model Input Data (Before Scaling):\n", input_data)

        input_scaled = scaler.transform(input_data)
        print("📊 Scaled Data (Before Prediction):\n", input_scaled)

        predicted_magnitude = model.predict(input_scaled)[0]

        if predicted_magnitude >= 6.0:
            risk_level = "High Earthquake Risk"
        elif predicted_magnitude >= 4.0:
            risk_level = "Moderate Earthquake Risk"
        else:
            risk_level = "Low Earthquake Risk"

        print("\n--- Earthquake Prediction ---")
        print(f"Predicted Magnitude: {predicted_magnitude:.2f}")
        print(f"⚠ Earthquake Risk Level: {risk_level}")

    else:
        print("❌ No recent earthquakes found for this location.")

predict_earthquake()


🔍 API Response: {'type': 'FeatureCollection', 'metadata': {'generated': 1739620830000, 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&limit=10&orderby=time', 'title': 'USGS Earthquakes', 'status': 200, 'api': '1.14.1', 'limit': 10, 'offset': 1, 'count': 10}, 'features': [{'type': 'Feature', 'properties': {'mag': 0.97, 'place': '3 km SSW of Anderson Springs, CA', 'time': 1739620604050, 'updated': 1739620700136, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc75133927', 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75133927&format=geojson', 'felt': None, 'cdi': None, 'mmi': None, 'alert': None, 'status': 'automatic', 'tsunami': 0, 'sig': 14, 'net': 'nc', 'code': '75133927', 'ids': ',nc75133927,', 'sources': ',nc,', 'types': ',nearby-cities,origin,phase-data,', 'nst': 9, 'dmin': 0.01924, 'rms': 0.02, 'gap': 135, 'magType': 'md', 'type': 'earthquake', 'title': 'M 1.0 - 3 km SSW of Anderson Springs, CA'}, 'geometry': {